# Run a training script with the Python SDK

You can use the Python SDK for Azure Machine Learning to submit scripts as jobs. By using jobs, you can easily keep track of the input parameters and outputs when training a machine learning model.

## Before you start

You'll need the latest version of the **azure-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [5]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.27.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-monitor-opentelemetry, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, pydash, pyjwt, pyyaml, six, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [49]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [50]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


Select Show for the Key field under key1.
Copy the value of the Key field to a notepad. You'll need to paste this value into the notebook later.
Copy the name of your storage account from the top of the page. The name should start with mlwdp100storage...
#access key, key1
MzYtM18zEmkiHiHgpwq1XccIB0F0WkZ+bYniIGWOLDgmycK7M0VK3VrdugGf+3GrECs254BiEQ3n+ASt1lt1XQ==
# name of storage account:
mlwhhndestorage7d141eba5

In [51]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/cidev20251103/code/Users/Mohamed.Outahajala/azure-ml-labs/Labs/02


In [52]:
import os
os.chdir('/home/azureuser/cloudfiles/code/Users/Mohamed.Outahajala/azure-ml-labs/Labs/02')
print("Working directory set to:", os.getcwd())

Working directory set to: /mnt/batch/tasks/shared/LS_root/mounts/clusters/cidev20251103/code/Users/Mohamed.Outahajala/azure-ml-labs/Labs/02


## Use the Python SDK to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [43]:
# make sure diabetes.csv is src folder
import pandas as pd
diabetes = pd.read_csv('./src/diabetes.csv')
print(diabetes.head())

   PatientID  Pregnancies  PlasmaGlucose  DiastolicBloodPressure  \
0    1354778            0            171                      80   
1    1147438            8             92                      93   
2    1640031            7            115                      47   
3    1883350            9            103                      78   
4    1424119            1             85                      59   

   TricepsThickness  SerumInsulin        BMI  DiabetesPedigree  Age  Diabetic  
0                34            23  43.509726          1.213191   21         0  
1                47            36  21.240576          0.158365   23         0  
2                52            35  41.511523          0.079019   23         0  
3                25           304  29.582192          1.282870   43         1  
4                27            35  42.604536          0.549542   22         0  


In [53]:
%%writefile src/diabetes-training.py
# import libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
import joblib

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('./diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# save the model
output_path = './outputs/model.pkl'
os.makedirs('./outputs', exist_ok=True)  # Ensure the outputs directory exists
joblib.dump(model, output_path)
print(f"Model saved to {output_path}")

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:,1])
print('AUC: ' + str(auc))

Overwriting src/diabetes-training.py


Run the cell below to submit the job that trains a classification model to predict diabetes. 

In [54]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster-dev",
    display_name="diabetes-pythonv2-train_regression-job",
    experiment_name="diabetes-training-regression-experiment",
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/green_line_blq9fsr82c?wsid=/subscriptions/0ded687b-997b-4159-acf3-9c82e7352f8c/resourcegroups/rg-hhn-dev/workspaces/mlw-hhn-dev-20251103&tid=e08b7eef-b501-4a67-9ed0-07e38bfccee7


In [55]:
print(returned_job.name + " job submitted successfully.")

green_line_blq9fsr82c job submitted successfully.


In [48]:
#check current directory
import os
print("Current working directory:", os.getcwd())

Current working directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/cidev20251103/code/Users/Mohamed.Outahajala/azure-ml-labs/Labs/02


In [56]:
# Step 1
# # Register the model from the job outputs
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# Get the job name
job_name = returned_job.name

model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/outputs/model.pkl",
    name="diabetes-logistic-regression",
    description="Logistic regression model for diabetes prediction",
    type=AssetTypes.CUSTOM_MODEL,
    tags={"accuracy": "0.774", "AUC": "0.848"}
)

registered_model = ml_client.models.create_or_update(model)
print(f"✅ Model registered: {registered_model.name} (version: {registered_model.version})")

✅ Model registered: diabetes-logistic-regression (version: 1)


Step 2 : Create a scoring script
Create a scoring script that will handle inference requests:

In [57]:
%%writefile src/score.py
import json
import joblib
import numpy as np
import os

def init():
    """
    This function is called when the container is initialized/started.
    """
    global model
    # Get the path to the registered model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    model = joblib.load(model_path)
    print("Model loaded successfully")

def run(raw_data):
    """
    This function is called for every invocation of the endpoint.
    """
    try:
        data = json.loads(raw_data)['data']
        data = np.array(data)
        result = model.predict(data)
        # Return predictions as JSON
        return json.dumps({"result": result.tolist()})
    except Exception as e:
        error = str(e)
        return json.dumps({"error": error})

Writing src/score.py


Step 3: Create Environment Configuration
Create a conda dependencies file

In [58]:
%%writefile src/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy
  - pip
  - scikit-learn
  - scipy
  - pip:
      - azureml-defaults
      - inference-schema[numpy-support]
      - joblib

Writing src/conda.yml


Step 4: Create a Managed Online Endpoint

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
from datetime import datetime

# Create a unique endpoint name
endpoint_name = f"diabetes-endpoint-{datetime.now().strftime('%m%d%H%M%f')}"

# Define the endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Endpoint for diabetes prediction",
    auth_mode="key"  # or "aml_token" for Azure AD authentication
)

# Create the endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print(f"✅ Endpoint created: {endpoint_name}")

Step 5: Create a Deployment

In [ ]:
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

deployment = ManagedOnlineDeployment(
    name="blue",  # Deployment name
    endpoint_name=endpoint_name,
    model=registered_model.id,
    code_configuration=CodeConfiguration(
        code="./src",
        scoring_script="score.py"
    ),
    environment=f"azureml://registries/azureml/environments/sklearn-1.0/versions/latest",
    instance_type="Standard_DS2_v2",
    instance_count=1
)

# Create the deployment
ml_client.online_deployments.begin_create_or_update(deployment).result()
print(f"✅ Deployment created: blue")

# Set traffic to 100% for this deployment
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()
print("✅ Traffic set to 100% for blue deployment")

Step 6: Test the Endpoint

In [ ]:
import json

# Sample test data (8 features)
test_data = {
    "data": [
        [2, 180, 74, 24, 21, 23.9, 1.488, 22],  # Sample patient data
        [0, 148, 58, 11, 179, 39.19, 0.160, 45]
    ]
}

# Invoke the endpoint
response = ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file=None,
    deployment_name="blue",
    request_body=json.dumps(test_data)
)

print("Prediction result:")
print(response)

Step 7: Get Endpoint Details and Use it Externally

In [ ]:
#step 7
# Get the endpoint details
endpoint_details = ml_client.online_endpoints.get(name=endpoint_name)

# Get the scoring URI
scoring_uri = endpoint_details.scoring_uri
print(f"Scoring URI: {scoring_uri}")

# Get the authentication key
keys = ml_client.online_endpoints.get_keys(name=endpoint_name)
primary_key = keys.primary_key
print(f"Primary Key: {primary_key}")

Step 8: Use the Endpoint with REST API

In [ ]:
#step 8
import requests

# Prepare the request
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {primary_key}'
}

test_data = {
    "data": [
        [2, 180, 74, 24, 21, 23.9, 1.488, 22]
    ]
}

# Make the prediction request
response = requests.post(scoring_uri, json=test_data, headers=headers)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")